## Creating original computer-generated music inspired by Bach's chorales

In this notebook, we create a model that will be able to generate original music after being trained on a large corpus comprised of J.S. Bach's chorale pieces.   

Some high-level features of this work: 
* This project is inspired by an exercise proposed in chapter 15 of the _Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow_ book. You can check out the author's implementation in [this notebook](https://github.com/ageron/handson-ml2/blob/master/15_processing_sequences_using_rnns_and_cnns.ipynb)      
* The model is based on an architecture consisting of stacked Gated Recurrent Units followed by a dense layer, regularized with droupout, and stabilized with layer normalization,          
* Training is performed in a many-to-many scheme: at each step, the model tries to predict the next note (as opposed to a many-to-one scheme, where the model would be exposed to a sequence of notes and try to predict the last one)   
* The dataset is available at [this link](https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/jsb_chorales/jsb_chorales.tgz) and contains _.csv_ files for each of the pieces    
* The dataset is split into train (229 pieces), validation (76 pieces) and test (77 pieces) partitions (yes, Bach was a tremendous workaholic!)   
* Each sample in the _.csv_ files contains 4 integer values ranging from 36 to 81 corresponding to 4 notes (one for each of the four voices) that are sung simultaneously, with an uniform note duration of half a beat      
* In order to train the model, each note is converted to a one-hot encoding to represent each of the possible notes, plus 0 for silence and -1 to indicate the end of a piece
* The dataset is augmented so that, before each training epoch, each of the pieces is randomly transposed by a number of steps in the range of -5 to 6 - this ensures that the model is equally exposed to musical patterns in all different keys   
* After training, the model is used to generate new music one note at a time by choosing each note randomly according to the probabilities generated by the softmax layer of the neural network      
* The original music is saved to midi files using the _mido_ package

### Importing packages

Besides the usual suspects, we also import the _mido_ package for creating and saving the _.mid_ files, as well as some functions from _os_ and _os.path_ to manipulate files and directories.   

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
from os import getcwd, listdir, mkdir
from os.path import join
import mido
import time

### Dataset preparation   

The dataset is downloaded from [this link](https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/jsb_chorales/jsb_chorales.tgz)

In [2]:
#### download/unzip dataset

Some helper functions are required to encode the music using one-hot encoding and decode the results from the softmax layer. Another helper function is defined to transpose an entire piece by a selected number of half steps.

In [3]:
# 81+6 and 36-5 due to transposing randomly between -5 half-steps and +6 half-steps

def one_hot_encoding(note):
    vector = np.zeros(((81+6)-(36-5)+3))
    if note == -1: # eof
        vector[0] = 1
    elif note == 0: # silence
        vector[1] = 1
    elif note >= (36-5) and note <= (81+6):
        vector[note-(36-5)+2] = 1
    else:
        raise Exception('invalid note, should be either -1 (eof), 0 (silence), or [36-5,81+6]; received {}'.format(note))
    return vector

def one_hot_decoding(vector):
    if vector.flatten()[0] == 1: # eof
        note = -1
    elif vector.flatten()[1] == 1: # silence
        note = 0
    else:
        note = vector.flatten().argmax() + (36-5) - 2
    return note

def transpose_note(note, interval):
    if note >= (36-5) and note <= (81+6):
        transposed = note + interval
    else:
        transposed = note
    return transposed

Next, some functions are defined to systematically prepared the train, valid and test datasets using tensorflow's data API.   

The make_dataset() function calls load_partition() to create a dataset consisting of a list of the files in each partition. This function then zips together each file and a randomly selected number of steps by which the piece will be transposed.   

Since the number of steps is randomly generated via a generator function, this quantity will be different for each piece and will be resampled at each epoch. This ensures that the model will be trained with each of the pieces transposed to several different keys, ensuring variability and improving generalization.   

After that, the parse_csv() function is called on each of the dataset entries (at this point, a file and a number of steps to transpose it by) via the flat_map() method. The parse_csv() function reads the _.csv_ file associated with one piece, decodes it, transposes all the notes and adds four end-of-file identifiers (-1).   

In [4]:
def generator_random_transpose():
    yield tf.random.Generator.from_non_deterministic_state().uniform(shape=[], minval=-5, maxval=6+1, dtype=tf.dtypes.int32).numpy()    

def parse_csv(file, interval): # added interval argument
    dataset_parsed = tf.data.TextLineDataset(file).skip(1)
    defaults = [0]*4
    dataset_parsed = dataset_parsed.map(lambda line: tf.stack(tf.io.decode_csv(line, defaults)))
    
    dataset_parsed = dataset_parsed.unbatch()    
    dataset_interval = tf.data.Dataset.from_tensor_slices([interval]).repeat()
    dataset_parsed = tf.data.Dataset.zip((dataset_parsed, dataset_interval))
    dataset_transposed = dataset_parsed.map(transpose_note)
    
    eof = tf.data.Dataset.from_tensor_slices([-1,-1,-1,-1])
    dataset_transposed = dataset_transposed.concatenate(eof)
    
    return dataset_transposed

def load_partition(partition='train'):
    filename = listdir(join(getcwd(), 'bach-chorales', partition))
    files = []
    for file in filename:
        files.append(join(getcwd(), 'bach-chorales', partition, file))
    dataset = tf.data.Dataset.list_files(files)
    
    dataset_random_transpose = tf.data.Dataset.from_generator(generator_random_transpose, output_types=tf.dtypes.int32)
    num_pieces = dataset.cardinality().numpy()
    dataset = tf.data.Dataset.zip((dataset, dataset_random_transpose.repeat(num_pieces)))
    
    dataset = dataset.flat_map(parse_csv) # flat_map ~ simple interleave
    return dataset

def make_dataset(partition='train'):
    dataset = list(load_partition(partition).as_numpy_iterator())
    x = dataset[:-1]
    y = dataset[1:]
    x_encoded = [one_hot_encoding(note) for note in x]
    y_encoded = [one_hot_encoding(note) for note in y]
    dataset = tf.data.Dataset.from_tensor_slices((x_encoded, y_encoded))
    return dataset

The make_dataset() function is called to prepare the train, valid and test sets. At this point, each dataset is composed of a sequence of single notes represented via a one-hot encoding. Splitting each into a number of different sequences and then into batches is done in the call to the model.fit() method, so that we can experiment with the sequence length and number samples per batch.   

In [5]:
train, valid, test = make_dataset('train'), make_dataset('valid'), make_dataset('test')

### Defining and training the model

The model is composed of a 3 stacked GRU units followed by a dense output layer with softmax activation. The inputs and outpus are of dimension 59 (corresponding to 57 possible notes plus the silence and end-of-file indications). The GRUs are twice as wide as the input and output dimension.

Because the model operates in a many-to-many scheme, every GRU layer returns its full sequence, which is used by the next layer.

The model benefits significantly from regularization, both in the form of dropout (present in both the memory and input channels of each GRU and before the dense layer) and L2 regularization.

Since this model is reasonably deep and deals with very long sequences, layer normalization is added to provide more stability during training.

We use keras' Sequential API to specify the model (as the model is linear, without any skip connections).

In [6]:
# model 2 -
#.47->.34 @train/ .65->.59 @valid/ .67->.62 @test
# with lr scheduling: .30 @train / .60 @valid / .65 @test
# with dropout .2, .2, .2: .38 @train / .58 @valid / .58 @test
# with dropout .3, .3, .2:

input_dim = 46 + 2 + 11 # 46 notes + 2 controls (silence & eof) + 11 transposing half-steps

rec_dropout, gru_dropout, dense_dropout, l2_penalty = .3, .3, .2, 0

model = keras.models.Sequential(name='model')
model.add(keras.layers.GRU(2 * input_dim, dropout = gru_dropout, recurrent_dropout = rec_dropout, 
                           input_shape = [None, input_dim], return_sequences = True, name='GRU1'))
model.add(keras.layers.GRU(2 * input_dim, dropout = gru_dropout, recurrent_dropout = rec_dropout, 
                           return_sequences = True, name='GRU2'))
model.add(keras.layers.GRU(2 * input_dim, dropout = gru_dropout, recurrent_dropout = rec_dropout, 
                           return_sequences = True, name='GRU3'))
model.add(keras.layers.Dropout(dense_dropout, name='Dropout1'))
model.add(keras.layers.Dense(input_dim, activation='softmax', 
                             kernel_regularizer = keras.regularizers.l2(l2 = l2_penalty), name='Dense1'))


# plan for model 2
# dropout before dense layer (duh) - .47->.34 @train/ .65->.59 @valid/ .67->.62 @test
# 4 bars of -1 - .48->.29 @train/ .65->.59 @valid/ .66->.59 @test / too many -1, songs too short, not very precise in outputing exactly 16
# remove dropout from output layer: .50->.?? @train/ .66->.?? @valid/ .69->.?? @test
# learning rate decay - did not improve results, but was able to approximate the results of 2 separate loops
# shuffle after 1st batch (shuffle the sequences so that GD does not receive biased sequences) - only if not using stateful RNN
# increase dropout on recurrent layers
# layer normalization on GRUs
# batch normalization on dense
# regularization parameters tuning
# change optimizers - decrease inertia
# stateful RNN 
    #(to learn longer paterns, maybe predict next entry without going through entire sequence) - 
    # not so simple, batching is complicated
    # create a stateful NN and copy the weights from the trained model, use it to generate new music faster
    # with a stateful RNN, try to implement MC dropout
# gradient clipping to avoid large peaks in loss functions - monitor gradients to see what is happening in those peaks
# consider adding dense layers (too few parameters in comparison to GRUs)


#: .??->.?? @train/ .??->.?? @valid/ .??->.?? @test

The model is trained with ADAM optimizer, learning rate scheduler function using power scheduling and reduce on plateau callback     

Power scheduling reduces to 1/2 after 1 cycle, then 1/3 after 2 cycles, then 1/4 after 3 cycles etc. Stronger reduction in the beginning is adequate, so the training can begin with a high learning rate that ensures that the easier patterns (such as the bias due to imbalanced classes) are recognized quicker. The reduce on plateau is more active mid-training, reducing by a fixed ratio every time there is no improvement to the validation loss for N consecutive epochs.

Tensorboard callback to save performance at the end of each epoch    

Early stopping and restore_best_weights to ensure that the final model contains the weights that minimized the loss function on the validation set    

The appropriate loss is categorical cross entropy - related to the model's predicted probability of the selected note. Accuracy is also measured.

The model contains around 240,000 trainable parameters.

Both the train and valid sets are batched in the call to the fit() method    

In [7]:
# First learning loop

sequence_length = 256
batch_size = 16

# setup learning rate scheduler

lr_0 = 0.01
lr_decay_rate = 1
lr_decay_step = 40 # 1 update per epoch

def lr_scheduler_fn(epoch, lr):
    #new_lr = lr_0 / (1 + lr_decay_rate*epoch/lr_decay_step) # not cool, depends on lr_0 instead of current lr, does not play well with reduce on plateau callback
    new_lr = lr * (1 + lr_decay_rate*epoch/lr_decay_step) / (1 + lr_decay_rate*(epoch+1)/lr_decay_step) # current lr time ratio between current and previous lr
    print('Epoch {} / current learning rate: {} / new learning rate: {}'.format(epoch,np.round(lr,6),np.round(new_lr,6)))
    return new_lr

lr_scheduler_callback = keras.callbacks.LearningRateScheduler(lr_scheduler_fn)

# setup for early stopping and tensorboard callbacks

if 'tb_logs' not in listdir(join(getcwd(), 'bach-chorales')):
    mkdir(join(getcwd(), 'bach-chorales', 'tb_logs'))

log_dir = join(getcwd(), 'bach-chorales', 'tb_logs', time.strftime("run_%Y_%m_%d-%H_%M_%S"))

tensorboard_callback = keras.callbacks.TensorBoard(log_dir = log_dir+'_loop_01', histogram_freq = 1)
early_stopping = keras.callbacks.EarlyStopping(patience=30, restore_best_weights=True, verbose=True)
lr_plateau_callback = keras.callbacks.ReduceLROnPlateau(factor=.3, min_lr = lr_0/100, patience=9, verbose=1)

# compile model, batch the datasets and train model

# PROBABLY SHOULDN'T BATCH THE VALIDATION SET! longer sequences tend to provide better prediction...
# also shouldn't randomly transpose validation set

optimizer = keras.optimizers.Adam(learning_rate = lr_0)
model.compile(optimizer=optimizer, loss = 'categorical_crossentropy', metrics = 'accuracy')
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
GRU1 (GRU)                   (None, None, 118)         63366     
_________________________________________________________________
GRU2 (GRU)                   (None, None, 118)         84252     
_________________________________________________________________
GRU3 (GRU)                   (None, None, 118)         84252     
_________________________________________________________________
Dropout1 (Dropout)           (None, None, 118)         0         
_________________________________________________________________
Dense1 (Dense)               (None, None, 59)          7021      
Total params: 238,891
Trainable params: 238,891
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(train.batch(sequence_length).batch(batch_size, drop_remainder=True).cache().prefetch(16),
                    epochs = 1000,
                    verbose = 1, # 1 : progress bar / 2 : one entry per epoch
                    validation_data = valid.batch(sequence_length).batch(batch_size, drop_remainder=True).cache().prefetch(1),
                    callbacks = [lr_scheduler_callback, lr_plateau_callback, tensorboard_callback, early_stopping])

epoch: 0 / current learning rate: 0.009999999776482582 / new learning rate: 0.009756097342909838
Epoch 1/1000
 1/54 [..............................] - ETA: 0s - loss: 4.0722 - accuracy: 0.0227WARNING:tensorflow:From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
54/54 [==============================] - 70s 1s/step - loss: 3.1946 - accuracy: 0.1437 - val_loss: 2.6699 - val_accuracy: 0.2028
epoch: 1 / current learning rate: 0.009756097570061684 / new learning rate: 0.009523809532679263
Epoch 2/1000
54/54 [==============================] - 70s 1s/step - loss: 2.1876 - accuracy: 0.3820 - val_loss: 1.4475 - val_accuracy: 0.5997
epoch: 2 / current learning rate: 0.009523809887468815 / new learning rate: 0.009302325936597447
Epoch 3/1000
54/54 [==============================] - 70s 1s/

54/54 [==============================] - 66s 1s/step - loss: 0.9032 - accuracy: 0.7491 - val_loss: 0.7660 - val_accuracy: 0.7947
epoch: 32 / current learning rate: 0.00555555522441864 / new learning rate: 0.005479451728193727
Epoch 33/1000
54/54 [==============================] - 66s 1s/step - loss: 0.8952 - accuracy: 0.7524 - val_loss: 0.7633 - val_accuracy: 0.7950
epoch: 33 / current learning rate: 0.005479451734572649 / new learning rate: 0.005405405089510856
Epoch 34/1000
54/54 [==============================] - 66s 1s/step - loss: 0.8950 - accuracy: 0.7520 - val_loss: 0.7601 - val_accuracy: 0.7952
epoch: 34 / current learning rate: 0.005405405070632696 / new learning rate: 0.0053333330030242605
Epoch 35/1000
54/54 [==============================] - 66s 1s/step - loss: 0.8915 - accuracy: 0.7529 - val_loss: 0.7590 - val_accuracy: 0.7962
epoch: 35 / current learning rate: 0.005333332810550928 / new learning rate: 0.005263157378833153
Epoch 36/1000
54/54 [=============================

54/54 [==============================] - 68s 1s/step - loss: 0.7994 - accuracy: 0.7726 - val_loss: 0.6727 - val_accuracy: 0.8129
epoch: 66 / current learning rate: 0.0037735842633992434 / new learning rate: 0.003738317120750652
Epoch 67/1000
54/54 [==============================] - 66s 1s/step - loss: 0.8008 - accuracy: 0.7726 - val_loss: 0.6668 - val_accuracy: 0.8141
epoch: 67 / current learning rate: 0.003738317172974348 / new learning rate: 0.003703703125076437
Epoch 68/1000
54/54 [==============================] - 67s 1s/step - loss: 0.8067 - accuracy: 0.7697 - val_loss: 0.6778 - val_accuracy: 0.8121
epoch: 68 / current learning rate: 0.003703703172504902 / new learning rate: 0.003669724244316784
Epoch 69/1000
54/54 [==============================] - 67s 1s/step - loss: 0.7977 - accuracy: 0.7719 - val_loss: 0.6757 - val_accuracy: 0.8124
epoch: 69 / current learning rate: 0.0036697243340313435 / new learning rate: 0.0036363632037219678
Epoch 70/1000
54/54 [==========================

epoch: 99 / current learning rate: 0.002877697115764022 / new learning rate: 0.0028571421363657074
Epoch 100/1000
54/54 [==============================] - 71s 1s/step - loss: 0.7418 - accuracy: 0.7845 - val_loss: 0.6356 - val_accuracy: 0.8200
epoch: 100 / current learning rate: 0.0028571421280503273 / new learning rate: 0.0028368787087024528
Epoch 101/1000
54/54 [==============================] - 73s 1s/step - loss: 0.7388 - accuracy: 0.7848 - val_loss: 0.6343 - val_accuracy: 0.8199
epoch: 101 / current learning rate: 0.0028368786443024874 / new learning rate: 0.002816900625680639
Epoch 102/1000
54/54 [==============================] - 70s 1s/step - loss: 0.7377 - accuracy: 0.7853 - val_loss: 0.6367 - val_accuracy: 0.8197
epoch: 102 / current learning rate: 0.002816900610923767 / new learning rate: 0.002797202005252971
Epoch 103/1000
54/54 [==============================] - 72s 1s/step - loss: 0.7361 - accuracy: 0.7856 - val_loss: 0.6336 - val_accuracy: 0.8200
epoch: 103 / current lear

In [ ]:
model.evaluate(train.batch(sequence_length).batch(batch_size, drop_remainder=True))

In [ ]:
model.evaluate(valid.batch(sequence_length).batch(batch_size, drop_remainder=True))

In [ ]:
model.evaluate(test.batch(sequence_length).batch(batch_size, drop_remainder=True))

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

### Generating new music

In [ ]:
# music generation

Length = 800 # number of new notes on all voices, actual piece length is Length/4
Temperature = 1.0 # not really temperature (which should be applied before softmax), but works similarly (smaller values are more adventurous, larger values are more conservative)

song = []
song_encoded = one_hot_encoding(-1).reshape((1,1,input_dim)).repeat(16, axis=1)
song_encoded.shape

new_note = -1
new_note_encoding = one_hot_encoding(new_note).reshape((1,1,input_dim))

# music generation loop

for _ in range(Length):
    p = model.predict(song_encoded)[:,-1,:].flatten() # the model is reading the entire chorale up to this point, which makes it very slow for long sequences. This should be improved, providing a limited number os samples for each new note or making the network remember the last state.
    p = p**Temperature / (p**Temperature).sum()
    
    out = np.random.choice(input_dim, p=p)
    new_note_encoding = np.zeros((1,1,input_dim))
    new_note_encoding[0,0,out] = 1
    new_note = one_hot_decoding(new_note_encoding)

    song.append(new_note)
    extended_song = np.zeros((song_encoded.shape[0], song_encoded.shape[1]+1, song_encoded.shape[2]))
    extended_song[:,:-1,:] = song_encoded
    extended_song[:,-1,:] = new_note_encoding
    song_encoded = extended_song

# print chorale as np array    
np.array(song).reshape((-1,4,4))

In [ ]:
# clip entries before beginning

first_note_position = 0
for position, note in enumerate(song):
    if note != -1:
        first_note_position = position
        break

new_chorale = np.array(song[first_note_position : -4 + (first_note_position % 4)]).reshape(-1,4)
        
# clip entries after eof

first_eof = (new_chorale.min(axis = 1) == -1).astype('int').argmax()
if first_eof > 0:
    new_chorale = new_chorale[:first_eof, :]
        
# convert finished product to data frame

new_chorale = pd.DataFrame(new_chorale, columns=['note1','note2','note3','note4'])
new_chorale

In [ ]:
# midi encoding with legato (attack only when new note is different from current one)

new_chorale_midi = mido.MidiFile(type=1)

time_unit = 360
instrument = 52 # choir aahs
volume = [50, 50, 60, 75] # volume for each channel

for channel in range(4):
    track = mido.MidiTrack()
    track.append(mido.Message('program_change', channel = channel, program = instrument, time = 0))
    previous_note = 0
    steps = 1
    
    for pos, note in enumerate(new_chorale.iloc[:, channel]):
        if note == previous_note:
            steps += 1
        if note != previous_note:
            if pos != 0:
                track.append(mido.Message('note_off', channel=channel, note=previous_note, time = steps * time_unit))
            track.append(mido.Message('note_on', channel=channel, note=note, velocity=volume[channel], time=0))
            previous_note = note
            steps = 1
            
    new_chorale_midi.tracks.append(track)
    
filename = 'new_chorale_' + time.strftime("_%Y_%m_%d-%H_%M_%S") + '.mid'
filepath = join(getcwd(), 'bach-chorales', 'new')

new_chorale_midi.save(join(filepath, filename))

### Sandbox

In [ ]:
# check what are the lowest and hightest notes in the corpus

#files = []
#partition = ['train', 'valid', 'test']

#for p in partition:
#    filename = listdir(join(getcwd(), 'bach-chorales', p))
#    for file in filename:
#        files.append(join(getcwd(), 'bach-chorales', p, file))

#pitch_range = []        

#for file in files:
#    df = pd.read_csv(file)
#    df = df.where(df != 0, other = None)
#    pitch_range.append((df.min().min(), df.max().max()))

#np.array(pitch_range).min(), np.array(pitch_range).max()

In [ ]:
# model 1 - .46->.34 @train / .66->.60 @valid / .67->.62 @test
#model = keras.models.Sequential()
#model.add(keras.layers.GRU(input_dim*2, dropout = .1, recurrent_dropout = .1, input_shape = [None, input_dim], return_sequences = True))
#model.add(keras.layers.GRU(input_dim*2, dropout = .1, recurrent_dropout = .1, return_sequences = True))
#model.add(keras.layers.GRU(input_dim*2, dropout = .1, recurrent_dropout = .1, return_sequences = True))
#model.add(keras.layers.Dense(input_dim, kernel_regularizer = keras.regularizers.l2(l2=0)))
#model.add(keras.layers.Dropout(.1))
#model.add(keras.layers.Activation('softmax'))

##### Second learning loop

In [ ]:
# Second learning loop

#learning_rate = 0.001 # 0.01 -> 0.001 -> 0.0001
#sequence_length = 256 # 256
#batch_size = 16 # 16 -> 16 -> 64

#optimizer = keras.optimizers.Adam(learning_rate = learning_rate)

#tensorboard_callback = keras.callbacks.TensorBoard(log_dir = log_dir+'_loop_02', histogram_freq = 1)

#model.compile(optimizer=optimizer, loss = 'categorical_crossentropy')

# PROBABLY SHOULDN'T BATCH THE VALIDATION SET! longer sequences tend to provide better prediction...

#history = model.fit(train.batch(sequence_length).batch(batch_size, drop_remainder=True).cache().prefetch(16),
#                    epochs = 1000,
#                    validation_data = valid.batch(sequence_length).batch(batch_size, drop_remainder=True).cache().prefetch(1),
#                    callbacks = [early_stopping, tensorboard_callback])

In [ ]:
#model.evaluate(train.batch(sequence_length).batch(batch_size, drop_remainder=True))

In [ ]:
#model.evaluate(valid.batch(sequence_length).batch(batch_size, drop_remainder=True))

In [ ]:
#model.evaluate(test.batch(sequence_length).batch(batch_size, drop_remainder=True))

In [ ]:
#plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])